# Desafio Técnico Etapa 1 - Aleon

Neste arquivo colocarei toda a preparação e passo a passo do desenvolvimento deste desafio, o qual é referente ao planejamento, escolha dos dados e o que se diz, *Storytelling* deste programa. Aqui contextualizarei o motivo da escolha dos dados e toda a narração e descrição do processo analítico.
```py
```

### Objetivo deste Desafio

1. Extrair dados de uma dessas API: Marvel, NASA, Gutendex, FDA, PokeAPI, Senado Federal
2. Formatar os dados desta API de forma tabular utilizando a biblioteca Pandas
3. Criar gráficos, análises, tabelas, sobre os dados extraídos, utilizando quaisquer bibliotecas (Seaborn. Matplotlib, etc). De 3 a 6 itens.
```py
```

### Minhas informações: Candidato João Pedro Fernandes

**Nome:** João Pedro Fernandes \
**Idade:** 20 anos, nasci em 08/01/2004 \
**Curso:** Bacharelado em Física Computacional na USP em São Carlos (Previsão de formatura: dez/2026) \
**Celular:** +55 16 99993-0311 \
**E-mail:** joao.cobblepot7@gmail.com \
**GitHub:** https://github.com/JonnCobblepot \
**LinkedIn:** [www.linkedin.com/in/joão-pedro-fernandes-1aa333268](https://www.linkedin.com/in/jo%C3%A3o-pedro-fernandes-1aa333268/) 
```py
```

### Primeiro Passo: Versionamento (Git)

Inicialmente, utilizei os comandos: `git init`, `git add .`, `git commit -m "descrição"`. Estes são responsáveis por inicializar o repositório, adicionar o arquivo na área de stage e depois salvar as alterações, respectivamente.

Após isso, criei no meu GitHub um repositório chamado Desafio-Tecnico-Etapa-1--Aleon, e assim vinculei com a minha máquina e enviei ao repositório do GitHub as alterações do projeto com os seguintes comandos: `git remote add origin https://github.com/JonnCobblepot/Desafio-Tecnico-Etapa-1--Aleon`, `git push origin main`.

O link do repositório que criei para desenvolver este projeto e ter o versionamento e controle de versão de seu projeto é o que se segue: \
https://github.com/JonnCobblepot/Desafio-Tecnico-Etapa-1--Aleon 

Assim, com tais comandos do Git, vou desenvolvendo passo a passo este desafio técnico.
```py
```

### Escolha da API MARVEL

Após verificar os possíveis sites com suas APIs, decidi escolher o API MARVEL. Primeiramente por um interesse pessoal, que sou muito fã da MARVEL e os super-heróis desde criança, e segundo porque entrei no site e vi que era de fácil acesso e visualização dos dados.

Com isso criei minha conta gratuita no _MyDisney_ referente ao site https://developer.marvel.com e li toda a documentação interativa da API. Como um grande fã, consegui ver ali a possibilidade de navegar entre os personagens, seus nomes e IDs, além de suas aparições em quadrinhos, desenhos, filmes e séries, e isso contribuiu para a minha escolha dessa API. Afinal também, é o gosto do cliente (no caso, eu mesmo), mesmo se fosse difícil eu ainda escolheria esse.

Após colocar minhas informações no site, obtive minha `public key` e `private key`, as quais são utilizadas para autenticação entre cliente e servidor para poder acessar os dados.

Em relação ao servidor, para testar os endpoints, seguindo o que está na documentação, precisa de 3 parâmetros:
- `apikey:` passagem para a Marvel da sua chave pública
- `ts:` registro de data e hora que se altera a cada solicitação
- `hash:` um parêmetro que utiliza das informações de chave privada e pública, além do ts, resumindo a autenticação. (o valor de hash é o resumo md5(ts+privateKey+publicKey)).

O parâmetro ts tem que ser calculado na hora da aplicação, e o hash deve ser calculado pela concatenação dos outros parâmetros. Para posteriormente ser usado numa chamada de exemplo, substituindo nos locais adequados:

`http://gateway.marvel.com/v1/public/comics?ts=COLOCA_TS&apikey=COLOCA_PUBLIC_KEY&hash=COLOCA_HASH`

Para poder obter os parâmetros ts e hash, calculei numa fórmula simples em Python e pesquisando vi que há uma biblioteca para hash, que facilitou na manipulação das informações:


In [3]:
import hashlib
import time

# Chaves publicas e privadas (coloquei as minhas para obter meus parêmtros e testar o endpoint)
PUBLIC_KEY = ""
PRIVATE_KEY = ""

# Valor do ts (timestamp) sendo a data e horário atual
ts = str(time.time())

# Soma para concatenar no hash
soma_hash = ts + PRIVATE_KEY + PUBLIC_KEY

# Gerando o hash pelo md5 visto na documentação da Marvel
hash = hashlib.md5(soma_hash.encode()).hexdigest()

# Printando os valores
print("ts:", ts)
print("Hash:", hash)

ts: 1734579218.2950974
Hash: 974d7340b1ccd17beac903a7c645bd17


Dentro dos endpoints disponíveis para pegar (GET), eu escolhi o dos personagens (characters) dado o acesso público: `/v1/public/characters`. No exemplo dado na documentação `http://gateway.marvel.com/v1/public/comics.....` eu então substituiria 'comics' por 'characters'. Assim, colocando isso e os parâmetros mencionados, é gerada a URL de requisição ao servidor.

Com isso, finalmente tenho acesso aos dados, e como o próprio nome já diz API = Application Programming Interface, conectamos ao sistema da Marvel para obter seus dados, autentificando através dos parêmetros para que somente devs autorizados o utilizem. A URL gerada é uma requisição HTTP, sendo o endereço que os servidores da Marvel de onde eu estou tirando os dados entende quem está fazendo isso (no caso, eu) e quais dados estou vendo deles e utilizando.

Antes de implementar o código para manipulação dos dados, é testada a URL no próprio navegador para ver se está retornando os dados. E tudo funcionou corretamente com os dados acessíveis. Não coloco aqui o printscreen da tela porque há informações sensíveis que não devem ser compartilhadas, como principalmente a chave privada.
```py
```

### Requisição na API da Marvel e Busca dos Dados

Agora é a etapa de realizar a requisição através do código. Então primeiramente, transformei a manipulação de parâmetros em uma função que será utilizada, retornando a URL da Requisição da API da Marvel. O código ficou da seguinte forma:

In [2]:
import hashlib
import time

def generate_parameter():

    # Chaves publicas e privadas (coloquei as minhas para obter meus parâmetros e testar o endpoint)
    PUBLIC_KEY = "...coloque aqui sua chave pública..."
    PRIVATE_KEY = "...coloque aqui sua chave privada..."

    # Valor do ts (timestamp) sendo a data e horário atual
    ts = str(time.time())

    # Soma para concatenar no hash
    soma_hash = ts + PRIVATE_KEY + PUBLIC_KEY

    # Gerando o hash pelo md5 visto na documentação da Marvel
    hash = hashlib.md5(soma_hash.encode()).hexdigest()

    URL = f'http://gateway.marvel.com/v1/public/characters?limit=5&ts={ts}&apikey={PUBLIC_KEY}&hash={hash}'

    return URL

# Printando os valores
#print("ts:", ts)
#print("Hash:", hash)
#print("apikey:", PUBLIC_KEY)
print("URL:", generate_parameter())

URL: http://gateway.marvel.com/v1/public/characters?limit=5&ts=1734579061.8112705&apikey=f957679ebe331b5cb3de7d6bc8531e39&hash=55df6a06e6bb1e6682f3186e97b2d36a


Eu apenas não coloquei as minhas chaves (PUBLIC_KEY e PRIVATE_KEY) aqui expostas, entretanto o código está testado e funcionando, obtendo a URL de requisição corretamente, conforme pode-se ver na saída que deixei desta célula do código, com a URL formada, a qual pode ser conferida requisição.